# Ethereum Yellow Paper超ふわっと要約

2018-12-10版Yellow Paperを要約しつつ重要なことを書き足す。進捗 : 6/39ページ

## 2 ブロックチェーンの定式化

Yellow Paperのセクション２では、すべての脱中央集権的ブロックチェーンに共通の構造を定式化している。
ブロックチェーンとは何かしらの内部状態を保持するデータベースであり、トランザクションとブロックの二重構造により内部状態（ステート）を変化させる。

- トランザクションによるステートの変更 - トランザクション$T$が$\Upsilon$関数を用いてステート$\sigma_t$を更新する。
$$ \sigma_{t+1} \equiv \Upsilon(\sigma_t, T)$$

- ブロックによるステートの変更 - ブロック$B$が$\Pi$関数を用いてステート$\sigma_t$を更新する。$B$はトランザクションの列とその他ブロック固有のパラメータで構成されている。$\Pi$はステートをブロック内の全トランザクションの分更新させたのち、マイニング報酬を与えるなどの最終処理関数$\Omega$を用いてステートを更新する。
$$ \sigma_{t+1} \equiv \Pi(\sigma_t, B) $$
$$ B \equiv (\dots, (T_0, T_1, \dots), \dots) $$
$$ \Pi(\sigma, B) \equiv \Omega(B, \dots\Upsilon(\Upsilon(\sigma, T_0), T_1)\dots) $$

Ethereumにおけるこれらのステート更新関数の具体的な定義はまた後ほど紹介する。

## 2.1 単位
続いてセクション２では通過単位を定義している。最小単位はWeiであり、それを元に次の表のように倍率を設定している。

|Multiplier|Name   |
|----------|-------|
|$10^0$      |Wei    |
|$10^{12}$     |Szabo  |
|$10^{15}$     |Finney |
|$10^{18}$     |Ether  |



In [1]:
class Blockchain:
    def __init__(self, init_state, tx_update, block_finalize):
        # input: state and tx
        # output: state
        self.tx_update = tx_update
        
        # input: block and state
        # output: state
        self.block_finalize = block_finalize
        
        # input: state and block
        # output: state
        self.block_update = lambda s, b: block_finalize(b, reduce(tx_update, b.transactions, s))
        
        self.state = init_state
    
    def apply_tx(tx):
        self.state = self.tx_update(self.state)
    
    def apply_block(tx):
        self.state = self.block_update(self.state)

wei = 1
szabo = 10**12
finney = 10**15
ether = 10**18

## 4.1 ステートの定義

ここではEthereumにおけるステートの定義を話す。

### 構成

Ethereumにおけるブロックチェーン全体のステート(ワールドステート、前項における$\sigma$)とは、アドレスとアカウントステートとの間の写像のこと。つまり、あるアドレスが与えられると、そのアドレスの状態（アカウントステート）を取得できるテーブルのことである。ここでアドレスは160ビットの整数であり、アカウントステートはRLPでエンコードされたデータである。

RLPとはデータ配列をバイト列に変換する関数であり、$\mathtt{RLP}$と表記する。

EthereumではハッシュにKeccak-256関数($\mathtt{KEC}$と表記)が用いられる。

ワールドステートはtrieという独自のハッシュ木で構成される。trieはハッシュ化することができ、keyとvalueの組$\mathfrak{I} = \{(k_0, v_0), (k_1, v_1), \dots\}$に対するハッシュを$\mathtt{TRIE}(\mathfrak{I})$と表す。

$a$をアカウントとした時のアカウントステート$\sigma[a]$は以下の4つで構成される。
- nonce: アカウントが今までに発行したトランザクション数。$\sigma[a]_n$と表記。
- balance: アカウントの残高。$\sigma[a]_b$と表記。
- storageRoot: アカウントの持つストレージのハッシュ。ストレージは256ビットの値同士の組を保持することができるもので、ワールドステートと同様にtrieである。$\sigma[a]_s$と表記。
- codeHash: アカウントの持つプログラムコードのハッシュ。$\sigma[a]_c$と表記。コードそのものを$b$と表記することがあり、その時は$\sigma[a]_c = \mathtt{KEC}(b)$である。

### データの処理

以下のように、$L_I^*$を使って$\sigma[a]_s$をtrieから取り出すことができる?（意味不明）
$$ \mathtt{TRIE}(L_I^*(\sigma[a]_s)) \equiv \sigma[a]_s$$

$\sigma[a]_c = \mathtt{KEC}(())$の時、すなわちアカウントのプログラムが空であるとき、そのアカウントは「普通の」アカウント、コントラクトでないアカウントである。

ワールドステートの簡約関数$L_S$を定義する。これはワールドステートから存在するアカウントのみを抽出し、配列に変換する関数である。
$$L_S(\sigma) \equiv \{p(a) : σ[a] \neq \emptyset \}$$
ここで、
$$p(a) \equiv (\mathtt{KEC}(a), \mathtt{RLP}((\sigma[a]_n, \sigma[a]_b, \sigma[a]_s, \sigma[a]_c)))$$
である。

ワールドステートのハッシュの際には、$L_S$と$\mathtt{TRIE}$を用いてハッシュする。以降、ワールドステートは次の形式に沿っているものとする。

1. ワールドステートに含まれる全てのアカウントは存在しないか、もしくはアドレス(trieのキー)が20バイトでそのアカウントステートが正しい形式かである。
$$\forall a : \sigma[a] = \emptyset \lor (a \in \mathbb{B}_{20} \land v(\sigma[a])$$
2. アカウントステートが正しい形式であるとは、nonceが256ビット整数、balanceが256ビット整数、storageRootが32バイト列、codeHashが32バイト列であることである。（256ビット整数と32バイト列は本質的に同じなので区別は趣味の問題である。）
$$v(x) \equiv x_n \in \mathbb{N}_{256} \land x_b \in \mathbb{N}_{256} \land x_s \in \mathbb{B}_{32} \land x_c \in \mathbb{B}_{32}$$

### アカウントの空・死

アカウント$a$が空であるとは、そのコードが空であり、そのnonceが0であり、その残高が0であることである。
$$\mathtt{EMPTY}(\sigma,a) \equiv \sigma[a]_c = \mathtt{KEC}(()) \land \sigma[a]_n = 0 \land \sigma[a]_b = 0$$

アカウント$a$が死んでいるとは、そのアカウントが存在しないか、空であることである。
$$\mathtt{DEAD}(\sigma,a) \equiv \sigma[a] = \emptyset \lor \mathtt{EMPTY}(\sigma,a)$$



In [2]:
def encode_address_account(t):
    (address, account) = t
    return (KEC(address), RLP(tuple(t)))

class State:
    def __init__(self):
        self.accounts = {}
    def serialize(self):
        return map(encode_address_account, self.accounts.items())
    
from collections import namedtuple
Address = namedtuple("Address", "nonce balance storageRoot codeHash")

## 4.2 トランザクションの定義

ここではトランザクションの定義を話す。

### 構成

トランザクションとはEthereumの外部から投稿される、署名された（ステートを更新させる）命令のことである。これは以下の要素からなる。

- nonce : トランザクションを送信したアカウントが今までに発行したトランザクション数。$T_n$と表記。
- gasPrice : 1gasあたりにかかるweiの量を指定する。$T_p$と表記。
- gasLimit : 計算に最大で何gas使えるかを指定する。この値は前払いであり、計算が行われる前から引き落とされる。$T_g$と表記。
- to : 送金先アドレス。160ビット数値。$T_t$と表記。
- value : アドレスtoに送金されるWeiの量。$T_v$と表記。
- v, r, s : トランザクションのECDSA署名。これを用いてトランザクションを作成したアカウント(sender)を計算する。$T_w, T_r, T_s$と表記。

コントラクト作成トランザクションの場合、以下の要素がある。

- init : コントラクトが作成されるときに使われるEVMコードを指定する。可変長バイト列。$T_i$と表記。

initを実行すると、bodyと呼ばれるコードが返される。bodyはコントラクトを呼ぶたびに実行されるコードで、コントラクトへの入力を処理する。つまり、initは「bodyというEVMプログラムを生成するEVMプログラム」ということになる。

コントラクトを呼び出すトランザクションの場合、以下の要素がある。

- data : コントラクトに入力するデータを指定する。可変長バイト列。$T_d$と表記。

### データの処理

実際にトランザクションを処理するために必要な関数を定義していく。

$S(T)$はトランザクションからそのトランザクションを作成したアカウントを計算する。$T_w, T_r, T_s$を使うと、ECDSAの性質から公開鍵を算出できる。

$L_T(T)$は$T_t$の有無からそのトランザクションがコントラクト呼び出し命令なのか作成命令なのかを判別し、適切な配列に並べる関数である。

$$
L_T(T)
\equiv
\left
\{
\begin{array}{l}
(T_n,T_p,T_g,T_t,T_v,T_\mathbf{i},T_w,T_r,T_s) & \text{if} \quad T_t = \emptyset \\
(T_n,T_p,T_g,T_t,T_v,T_\mathbf{d},T_w,T_r,T_s) & \text{otherwise}
\end{array}
\right.
$$

以降、

- nonce, gasPrice, gasLimit, value, r, sは256ビット数値
- vは5ビット数値
- data, initは可変長バイト列
- toは20バイト列　もしくは　空文字列

とする。toを空文字列にした場合、トランザクションはコントラクト作成命令になる。

$$
\begin{equation}
\begin{array}[t]{lclclc}
T_{\mathrm{n}} \in \mathbb{N}_{256} & \wedge & T_{\mathrm{v}} \in \mathbb{N}_{256} & \wedge & T_{\mathrm{p}} \in \mathbb{N}_{256} & \wedge \\
T_{\mathrm{g}} \in \mathbb{N}_{256} & \wedge & T_{\mathrm{w}} \in \mathbb{N}_5 & \wedge & T_{\mathrm{r}} \in \mathbb{N}_{256} & \wedge \\
T_{\mathrm{s}} \in \mathbb{N}_{256} & \wedge & T_{\mathbf{d}} \in \mathbb{B} & \wedge & T_{\mathbf{i}} \in \mathbb{B}
\end{array}
\end{equation}
$$

$$
\begin{equation}
T_{\mathbf{t}} \in \begin{cases} \mathbb{B}_{20} & \text{if} \quad T_{\mathrm{t}} \neq \varnothing \\
\mathbb{B}_{0} & \text{otherwise}\end{cases}
\end{equation}
$$

## 4.3 ブロックの定義

ここではブロックの定義について話す。

### 構成

Ethereumではブロックタイムを短縮するため、一本鎖に繋がれたブロック以外にも報酬が行き渡るようになっている。つまり、ある程度はフォークのブロックにもマイニング報酬が支払われるということだ。

ちなみに2019/3/2現在、EthereumではまだPoSは実装されていなく、完全にPoWである。このことは次に紹介するブロック構造を見てもよくわかる。

ブロックヘッダは以下で構成される。

- parentHash: 親のブロックのハッシュ。$H_{\mathrm{p}}$と表記。
- ommersHash: 「『自分のブロックとおなじ親の親』を親に持つブロックたち」（ommers）を総合したハッシュ。$H_{\mathrm{o}}$と表記。
- beneficiary: マイニング報酬を受け取るアドレス。$H_{\mathrm{c}}$と表記。
- stateRoot: ワールドステートのハッシュ。前述の通りワールドステートはアカウントとアカウントステートのtrie。$H_{\mathrm{r}}$と表記。
- transactionsRoot: 全トランザクションのハッシュ。リストをtrieにしてハッシュする。$H_{\mathrm{t}}$と表記。
- receiptsRoot: 全レシートのハッシュ。リストをtrieにしてハッシュする。$H_{\mathrm{e}}$と表記。
- logsBloom: 全ログのハッシュ。リストをtrieにしてハッシュする。$H_{\mathrm{b}}$と表記。
- difficulty: ブロックのマイニング難易度。$H_{\mathrm{d}}$と表記。
- number: ブロック番号。$H_{\mathrm{i}}$と表記。
- gasLimit: ブロック全体の最大gas消費量（最大計算量）。$H_{\mathrm{l}}$と表記。
- gasUsed: ブロック全体のgas消費量（計算量）。$H_{\mathrm{g}}$と表記。
- timestamp: ブロックのタイムスタンプ。$H_{\mathrm{s}}$と表記。
- extraData: 32バイト以内の恣意的な文字列。$H_{\mathrm{x}}$と表記。
- mixHash, nonce: 所定の量の計算量を消費（マイニング）したことを証明する。mixHash=$H_{\mathrm{m}}$、nonce=$H_{\mathrm{n}}$と表記。

レシートはトランザクションが実行された証明のことである。一つのトランザクションにつき一つ生成される。

ログはトランザクションやアカウントの検索を行うためのマーキングデータである。コントラクト内で発生させることができる。（ログを記録するEVM命令が存在する。）

ommersは人間で言うところの"おじ"や"おば"に当たる。

ブロック$B$はブロックヘッダ$B_H$、トランザクションのリスト$B_T$、ommerのヘッダのリスト$B_U$からなる。レシートやログはトランザクションから計算されるため、この３つで十分である。
$$B \equiv (B_H,B_T,B_U)$$


### 4.3.1 レシートとログ
レシート$R$は各トランザクションから算出される、支払い証明や検索のために使うデータのことである。これは以下の要素からなる。

- $R_\mathrm{u}$ : そのトランザクションが所属するブロックにおいて、そのトランザクションが処理された直後までに使用されたgasの累計。
- $R_\mathrm{l}$ : そのトランザクションが実行される時に生成された全てのログ。
- $R_\mathrm{b}$ : そのログから生成されたBloomフィルター。
- $R_\mathrm{z}$ : トランザクションのステータスコード。

$$R \equiv (R_\mathrm{u}, R_\mathrm{b}, R_\mathrm{l}, R_\mathrm{z})$$

今後のデータ処理のため、以下の関数を定義する。最初に0で埋まった256バイト列を追加したのは、以前のプロトコルに存在したpre-transaction stateというパラメータを置き換えるためである。

$$L_R(R) \equiv (0 \in \mathbb{B}_{256}, R_\mathrm{u}, R_\mathrm{b}, R_\mathrm{l})$$

ステータスコード$ R_\mathrm{z}$は負でない整数とする。

$$ R_\mathrm{z} \in \mathbb{N}$$

gas累計$ R_\mathrm{u} $は正の整数であり、Bloomフィルター$ R_\mathrm{b} $は256バイト列とする。

$$ R_\mathrm{u} \in \mathbb{N} \land R_\mathrm{b} \in \mathbb{B}_{256}$$

$R_\mathrm{l}$はログの配列である。

$$R_\mathrm{l} = (O_0, O_1, ...)$$

ログ$O$はコントラクト内で発生させることができる、トランザクションやアカウントの検索を行うためのマーキングデータである。これは以下の要素からなる。

- $O_\mathrm{a}$ : ログ発生者のアドレス
- $O_\mathbf{t}$（トピック）: 32バイト列のリスト
- $O_\mathbf{d}$ : 可変長のバイト列

$$
O \equiv (O_{\mathrm{a}}, ({O_{\mathbf{t}}}_0, {O_{\mathbf{t}}}_1, ...), O_{\mathbf{d}})
$$

$$
O_{\mathrm{a}} \in \mathbb{B}_{20} \quad \wedge \quad \forall x \in O_{\mathbf{t}}: x \in \mathbb{B}_{32} \quad \wedge \quad O_{\mathbf{d}} \in \mathbb{B}
$$

Bloomフィルター関数$M$を定義する。これは一つのログを256バイトにハッシュ化するものである。ただし、その際に$O_\mathbf{d}$は無視される。まず、$O$のアドレス$O_\mathrm{a}$と全てのトピック${O_\mathbf{t}}_0, {O_\mathbf{t}}_1, ...$をそれぞれ「3つの0~2047までの値」にし、その値が指すビットを全て1にした256バイトの値を返す。

$\mathbf{x}$に対する「3つの0~2047までの値」は、$\mathtt{KEC}(\mathbf{x})$の0~1バイト目、2~3バイト目、4~5バイト目をそれぞれ2048で割ったあまりと定義する。式にすると以下になる。ここでは$\bigvee$はビットORを表す。

$$
M(O) \equiv \bigvee_{x \in \{O_{\mathrm{a}}\} \cup O_{\mathbf{t}}} \big( M_{3:2048}(x) \big)
$$

$$
\begin{array}{rcl}
M_{3:2048}(\mathbf{x}: \mathbf{x} \in \mathbb{B}) & \equiv & \mathbf{y}: \mathbf{y} \in \mathbb{B}_{256} \quad \text{where:}\\
\mathbf{y} & = & (0, 0, ..., 0) \quad \text{except:}\\
\forall_{i \in \{0, 2, 4\}}&:& \mathcal{B}_{m(\mathbf{x}, i)}(\mathbf{y}) = 1\\
m(\mathbf{x}, i) &\equiv& \mathtt{KEC}(\mathbf{x})[i, i + 1] \bmod 2048
\end{array}
$$

ここで$\mathcal{B}_j(\mathbf{x})$は$\mathbf{x}$の$j$ビット目を表す。

Bloomフィルターとはその名の通りフィルターであり、検索のために使うデータである。例えばあるトピックを持つ（可能性がある）トランザクションを全て列挙したい時、そのトピックに対する$M_{3:2048}(x)$関数を呼ぶと、2048ビットの中で3つの箇所だけが1になっている数が出力される。その数と同じ箇所が1になっているBloomフィルターを持つトランザクションに検索したいトピックがある可能性が高い。そうやって大まかな絞り込みをした後に、実際にログに検索をかければ良いということになる。

式ではわかりにくいので以下にコードを示す。

In [3]:
def bloom_filter(log):
    address = log.address
    topics = log.topics
    out = 0
    
    # アドレスと全トピックに対して...
    for x in [address] + topics:
        # ハッシュを計算
        h = KEC(x)
        
        # 計算したハッシュの0~1  2~3   4~5バイト目を読み取る
        for i in [0, 2, 4]:
            # ビットを1にする位置を計算
            m = (ord(h[i])*256 + ord(h[i+1])) % 2048
            
            # 計算したビットを1にする
            out = out | (2**m)
    
    # 数値を256バイトに変換
    return out.to_bytes(256, "big")

### 4.3.2 ブロックの検証

ブロックは特定の形式を満たさなければならない。Ethereumノードはブロックに含まれているトランザクション$B_T$やommerのヘッダ$B_U$を使ってステート$\sigma$やハッシュなどを計算して、その結果をブロックヘッダと照合する。同時にこれはマイナーがブロックヘッダを計算するときの計算式にもなっている。

各ハッシュは以下で定義される。

- ワールドステートのハッシュ$H_{\mathrm{r}}$ : 前回のステート$\boldsymbol{\sigma}$を$\Pi$で更新し、$L_S$と$\mathtt{TRIE}$でハッシュ化。
- ommersのヘッダのハッシュ$H_{\mathrm{o}}$ : $L_H^*$でommerのヘッダ$B_{\mathbf{U}}$をRLPにしたのち、$\mathtt{KEC}$でハッシュ化。
- 全トランザクションのハッシュ$H_{\mathrm{t}}$ : トランザクション番号$i$とトランザクションの内容を符号化したもの$L_{T}(B_{\mathbf{T}}[i])$のペアを$\mathtt{TRIE}$でハッシュ化。
- 全レシートのハッシュ$H_{\mathrm{e}}$ : トランザクション番号$i$とレシートの内容を符号化したもの$L_{R}(B_{\mathbf{R}}[i]))$のペアを$\mathtt{TRIE}$でハッシュ化。
- 全ログのハッシュ$H_{\mathrm{b}}$ : 全てのBloomフィルターをビットORする。

$$
\begin{equation}
\begin{array}[t]{lclc}
H_{\mathrm{r}} &\equiv& \mathtt{TRIE}(L_S(\Pi(\boldsymbol{\sigma}, B))) & \wedge \\
H_{\mathrm{o}} &\equiv& \mathtt{KEC}(\mathtt{RLP}(L_H^*(B_{\mathbf{U}}))) & \wedge \\
H_{\mathrm{t}} &\equiv& \mathtt{TRIE}(\{\forall i < \lVert B_{\mathbf{T}} \rVert, i \in \mathbb{N}: &\\&& \quad\quad p (i, L_{T}(B_{\mathbf{T}}[i]))\}) & \wedge \\
H_{\mathrm{e}} &\equiv& \mathtt{TRIE}(\{\forall i < \lVert B_{\mathbf{R}} \rVert, i \in \mathbb{N}: &\\&& \quad\quad p(i, L_{R}(B_{\mathbf{R}}[i]))\}) & \wedge \\
H_{\mathrm{b}} &\equiv& \bigvee_{\mathbf{r} \in B_{\mathbf{R}}} \big( \mathbf{r}_{\mathrm{b}} \big)
\end{array}
\end{equation}
$$

ここで今までにデータ処理用に定義してきた関数が登場する。

- $L_S$ : ワールドステートから存在するアカウントのみを抽出し、$(\mathtt{KEC}(a), \mathtt{RLP}((\sigma[a]_n, \sigma[a]_b, \sigma[a]_s, \sigma[a]_c)))$の形式の配列にエンコードする。
- $L_T$ : トランザクションがコントラクト呼び出し命令なのか作成命令なのかを判別し、適切な配列に並べる。
- $L_R$ : 最初の256バイトは全て0で、$R_\mathrm{u}, R_\mathrm{b}, R_\mathrm{l}$が続く。

$L_H^*$は次項で定義する。

また、$p$は値のペアをRLP形式に変換する関数である。

$$
p(k, v) \equiv \big( \mathtt{RLP}(k), \mathtt{RLP}(v) \big)
$$
